## Note this notebook is same as other grab categoricals just uses label encoding instead of one hot for the main dataframe that has most of key categorical columns(income type, family, property type etc.) I wanted to see if this would make a difference for the tree based methods RF classifier, xgboost classifier, and lgbm

In [1]:
import pandas as pd
import numpy as np
import feather
from sklearn.metrics import (auc,accuracy_score, average_precision_score,
                             balanced_accuracy_score,roc_auc_score,
                            roc_curve)
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.model_selection import train_test_split,RandomizedSearchCV,KFold
import matplotlib.pyplot as plt
import re






In [2]:
!ls

'Clean Data .ipynb'			  df_test_processed.feather
'Grab Categoricals(Label Encode).ipynb'   df_train_one_hot.feather
'Grab Categoricals(one_hot).ipynb'	  df_train_processed.feather
 HomeCredit_columns_description.csv	  df_train_test_app_cleaned.feather
 POS_CASH_balance.csv			  home-credit-default-risk.zip
 application_test.csv			  home_credit_explore.ipynb
 application_train.csv			  installments_payments.csv
 bureau.csv				  previous_application.csv
 bureau_balance.csv			  sample_submission.csv
 credit_card_balance.csv		  xgboost_w_out_clean_null.pkl
 df_test_one_hot.feather


In [3]:
list_csvs = !ls *.csv

In [4]:
list_csvs

['HomeCredit_columns_description.csv',
 'POS_CASH_balance.csv',
 'application_test.csv',
 'application_train.csv',
 'bureau.csv',
 'bureau_balance.csv',
 'credit_card_balance.csv',
 'installments_payments.csv',
 'previous_application.csv',
 'sample_submission.csv']

In [5]:
import csv

In [6]:
for file in list_csvs[1:]:
    df = pd.read_csv(file)
    print(f"{file}: {df.shape}")

POS_CASH_balance.csv: (10001358, 8)
application_test.csv: (48744, 121)
application_train.csv: (307511, 122)
bureau.csv: (1716428, 17)
bureau_balance.csv: (27299925, 3)
credit_card_balance.csv: (3840312, 23)
installments_payments.csv: (13605401, 8)
previous_application.csv: (1670214, 37)
sample_submission.csv: (48744, 2)


In [7]:
for file in list_csvs[1:]:
    df = pd.read_csv(file)
    print(f"DF NAME: {file}ii")
    print(f"{df.shape}")
    for col in df.columns:
        print(f"{col}: {df[col].isnull().sum()}")
    print("\n")

DF NAME: POS_CASH_balance.csvii
(10001358, 8)
SK_ID_PREV: 0
SK_ID_CURR: 0
MONTHS_BALANCE: 0
CNT_INSTALMENT: 26071
CNT_INSTALMENT_FUTURE: 26087
NAME_CONTRACT_STATUS: 0
SK_DPD: 0
SK_DPD_DEF: 0


DF NAME: application_test.csvii
(48744, 121)
SK_ID_CURR: 0
NAME_CONTRACT_TYPE: 0
CODE_GENDER: 0
FLAG_OWN_CAR: 0
FLAG_OWN_REALTY: 0
CNT_CHILDREN: 0
AMT_INCOME_TOTAL: 0
AMT_CREDIT: 0
AMT_ANNUITY: 24
AMT_GOODS_PRICE: 0
NAME_TYPE_SUITE: 911
NAME_INCOME_TYPE: 0
NAME_EDUCATION_TYPE: 0
NAME_FAMILY_STATUS: 0
NAME_HOUSING_TYPE: 0
REGION_POPULATION_RELATIVE: 0
DAYS_BIRTH: 0
DAYS_EMPLOYED: 0
DAYS_REGISTRATION: 0
DAYS_ID_PUBLISH: 0
OWN_CAR_AGE: 32312
FLAG_MOBIL: 0
FLAG_EMP_PHONE: 0
FLAG_WORK_PHONE: 0
FLAG_CONT_MOBILE: 0
FLAG_PHONE: 0
FLAG_EMAIL: 0
OCCUPATION_TYPE: 15605
CNT_FAM_MEMBERS: 0
REGION_RATING_CLIENT: 0
REGION_RATING_CLIENT_W_CITY: 0
WEEKDAY_APPR_PROCESS_START: 0
HOUR_APPR_PROCESS_START: 0
REG_REGION_NOT_LIVE_REGION: 0
REG_REGION_NOT_WORK_REGION: 0
LIVE_REGION_NOT_WORK_REGION: 0
REG_CITY_NOT_LIVE_CI

## Label Encoding has some issues with null so need to fix that before making function to label encode

In [39]:
df_train = pd.read_csv("application_train.csv")

In [41]:
for col in df_train.columns: 
    if df_train[col].dtype=="O":
        print(f"{col}: {df_train[col].isnull().sum()}")
del df_train

NAME_CONTRACT_TYPE: 0
CODE_GENDER: 0
FLAG_OWN_CAR: 0
FLAG_OWN_REALTY: 0
NAME_TYPE_SUITE: 1292
NAME_INCOME_TYPE: 0
NAME_EDUCATION_TYPE: 0
NAME_FAMILY_STATUS: 0
NAME_HOUSING_TYPE: 0
OCCUPATION_TYPE: 96391
WEEKDAY_APPR_PROCESS_START: 0
ORGANIZATION_TYPE: 0
FONDKAPREMONT_MODE: 210295
HOUSETYPE_MODE: 154297
WALLSMATERIAL_MODE: 156341
EMERGENCYSTATE_MODE: 145755


## I am ging to try and replace these nulls with the non-null distribution of the given column

In [68]:
def make_categorical_main(df):
    le = LabelEncoder()
    cat_columns = [col for col in df.columns if df[col].dtype=="O"]
    old_columns = [col for col in df.columns]
    for col in cat_columns:
        dist = df[col].value_counts(normalize=True,dropna=True)
        missing = df[col].isnull()
        df.loc[missing,col] = np.random.choice(dist.index,
                                               size=len(df[missing]),
                                              p=dist.values)
        df[col] = pd.Categorical(df[col])
        df[col]=le.fit_transform(df[col])
    
    new_cats = [col for col in df.columns if col not in old_columns]
    return df,new_cats

In [43]:
def make_categorical(df):
    cat_columns = [col for col in df.columns if df[col].dtype=="O"]
    old_columns = [col for col in df.columns]
    df = pd.get_dummies(df,drop_first=True,dummy_na=True)
    new_cats = [col for col in df.columns if col not in old_columns]
    return df,new_cats

## GEt an idea of what numeric columns maybe have odd values

In [44]:
df_train = pd.read_csv("application_train.csv")
df_test = pd.read_csv("application_test.csv")
df_test.insert(1,"TARGET",np.nan)
df_tot = pd.concat([df_train,df_test])

In [45]:
df_tot.shape

(356255, 122)

In [46]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
for col in df_tot.select_dtypes(include=numerics).columns:
    print(f"{col} mean: {df_tot[col].mean()}, min: {df_tot[col].min()} max: {df_tot[col].max()}")
    
    
    
    

SK_ID_CURR mean: 278128.0, min: 100001 max: 456255
TARGET mean: 0.08072881945686496, min: 0.0 max: 1.0
CNT_CHILDREN mean: 0.4143155885531431, min: 0 max: 20
AMT_INCOME_TOTAL mean: 170116.05986028264, min: 25650.0 max: 117000000.0
AMT_CREDIT mean: 587767.4137527895, min: 45000.0 max: 4050000.0
AMT_ANNUITY mean: 27425.56065650625, min: 1615.5 max: 258025.5
AMT_GOODS_PRICE mean: 528019.9978004197, min: 40500.0 max: 4050000.0
REGION_POPULATION_RELATIVE mean: 0.020917030374318486, min: 0.000253 max: 0.072508
DAYS_BIRTH mean: -16041.248841419769, min: -25229 max: -7338
DAYS_EMPLOYED mean: 64317.23141289245, min: -17912 max: 365243
DAYS_REGISTRATION mean: -4983.593527225349, min: -24672.0 max: 0.0
DAYS_ID_PUBLISH mean: -3002.07116251, min: -7197 max: 0
OWN_CAR_AGE mean: 12.023741054753996, min: 0.0 max: 91.0
FLAG_MOBIL mean: 0.9999943860437046, min: 0 max: 1
FLAG_EMP_PHONE mean: 0.818497985993179, min: 0 max: 1
FLAG_WORK_PHONE mean: 0.20009824423516864, min: 0 max: 1
FLAG_CONT_MOBILE mean: 0.

### Days Employed has a mean of 175 years so this must be changed 

In [47]:
df_tot['DAYS_EMPLOYED'].value_counts()[0:5]

 365243    64648
-200         175
-199         171
-212         165
-224         163
Name: DAYS_EMPLOYED, dtype: int64

#### Going to make a function that handles the negative days as this found to be a  recurring occurence in later investigation of other dfs

In [48]:
def make_days_positive(df):
    day_reg = re.compile("DAYS|MONTHS")
    for col in df.columns:
        if re.search(day_reg, col):
            if df[col].mean()<0:
                df[col]*=-1
    return df

#### Also on later inspection, lots of dataframes had max value of 365423 in a column that had to do with time so I will also write a function to replace these

In [49]:
def replace_max_erroneous_days(df):
    day_reg = re.compile("DAY|MONTHS")
    for col in df.columns:
        if re.search(day_reg, col):
            if (df[col].max()==365423.0):
                df[col].replace(365243.0,
                                df_apps['DAYS_FIRST_DRAWING'].median(),
                                inplace=True)
            elif (df[col].max()==365423):
                df[col].replace(365243,
                                df_apps['DAYS_FIRST_DRAWING'].median(),
                                inplace=True)
    return df

In [50]:
def process_train_test():
    df = pd.read_csv("application_train.csv")
    df_test = pd.read_csv("application_test.csv")
    df_test.insert(1,"TARGET",np.nan)
    df_tot = pd.concat([df,df_test])
    df_tot, cat_cols = make_categorical_main(df_tot)
    cat_cols.insert(0,"SK_ID_CURR")
    df_tot['Income_Credit_Ratio'] = (df_tot['AMT_INCOME_TOTAL'] 
                                     / df_tot['AMT_CREDIT'])
    df_tot['Income_per_Person'] = (df_tot['AMT_INCOME_TOTAL']/
                                        df_tot['CNT_FAM_MEMBERS'])
    df_tot['Annuity_Income_Ratio'] = (df_tot['AMT_ANNUITY']/
                                     df_tot['AMT_INCOME_TOTAL'])
    df_tot = make_days_positive(df_tot)
    replace_max_erroneous_days(df_tot)
    #median_days_employed = df_tot['DAYS_EMPLOYED'].median()
    #df_tot.loc[df_tot['DAYS_EMPLOYED']==365243,"DAYS_EMPLOYED"]=median_days_employed
    #df_app_train = df_tot.iloc[0:307511,:].copy()
    #df_app_test = df_tot.iloc[307511:,:].copy()
    del df, df_test
    return df_tot
    
    

## At a quick glance the rest of the numeric columns do not seem to have very strange values so I will keep them as is here

In [51]:
df_bur = pd.read_csv("bureau.csv")

In [52]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
for col in df_bur.select_dtypes(include=numerics).columns:
    print(f"{col} mean: {df_bur[col].mean()}, min: {df_bur[col].min()} max: {df_bur[col].max()}")
    
    
    
    

SK_ID_CURR mean: 278214.9336447553, min: 100001 max: 456255
SK_ID_BUREAU mean: 5924434.489031873, min: 5000000 max: 6843457
DAYS_CREDIT mean: -1142.107684680045, min: -2922 max: 0
CREDIT_DAY_OVERDUE mean: 0.8181665645165425, min: 0 max: 2792
DAYS_CREDIT_ENDDATE mean: 510.51736230309615, min: -42060.0 max: 31199.0
DAYS_ENDDATE_FACT mean: -1017.43714806862, min: -42023.0 max: 0.0
AMT_CREDIT_MAX_OVERDUE mean: 3825.417661273144, min: 0.0 max: 115987185.0
CNT_CREDIT_PROLONG mean: 0.00641040579622332, min: 0 max: 9
AMT_CREDIT_SUM mean: 354994.59191768744, min: 0.0 max: 585000000.0
AMT_CREDIT_SUM_DEBT mean: 137085.11995216075, min: -4705600.32 max: 170100000.0
AMT_CREDIT_SUM_LIMIT mean: 6229.514980300509, min: -586406.115 max: 4705600.32
AMT_CREDIT_SUM_OVERDUE mean: 37.91275774165883, min: 0.0 max: 3756681.0
DAYS_CREDIT_UPDATE mean: -593.7483197663986, min: -41947 max: 372
AMT_ANNUITY mean: 15712.757700078355, min: 0.0 max: 118453423.5


In [53]:
del df_bur

## Note: With regards to the aggregations for each supplemntary df, I will try several different types of aggregations and then see if any prove to play a role when I do rf.feature_importances_ or xgboost.feature_importances_, also based on the main df, columns in days  happened to be negative so I will check for this in the other dataframes and correct accordingly 

In [54]:
def process_bureau():
    df_bur = pd.read_csv("bureau.csv")
    df_bur,new_cats = make_categorical(df_bur)
    df_bur.drop("SK_ID_BUREAU",axis=1,inplace=True)
    aggs = {
        "DAYS_CREDIT": ["mean","max","min"],
        "DAYS_CREDIT_ENDDATE": ['mean',"max","min"],
        "CREDIT_DAY_OVERDUE": ["mean","max","min"],
        "CNT_CREDIT_PROLONG": ["mean","max","min"],
        "AMT_CREDIT_MAX_OVERDUE": ["mean","max","min"],
        "AMT_CREDIT_SUM": ["mean","max","min"],
        "AMT_CREDIT_SUM_DEBT": ["mean","max","min"],
        "AMT_CREDIT_SUM_LIMIT": ["mean","max","min"],
        "AMT_CREDIT_SUM_OVERDUE": ["mean","max","min"],
        "DAYS_CREDIT_UPDATE": ["mean","max","min"],
        "AMT_ANNUITY": ["mean","max","min"],   
    }
    df_bur = make_days_positive(df_bur)
    df_bur = replace_max_erroneous_days(df_bur)
    for cat in new_cats: 
           aggs[cat]=['mean']
    #df_cat_process = df_bur[['SK_ID_CURR']+new_cats].copy()
    #df_bur.drop(new_cats, axis=1,inplace=True)
    bureau_agg = df_bur.groupby("SK_ID_CURR").agg(aggs)
    bureau_agg.columns = bureau_agg.columns.map('_'.join)
    bureau_agg.reset_index(inplace=True)
    del df_bur
    return bureau_agg

### Process Credit Card Review Info

In [55]:
df_credit = pd.read_csv("credit_card_balance.csv")
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
for col in df_credit.select_dtypes(include=numerics).columns:
    print(f"{col} mean: {df_credit[col].mean()}, min: {df_credit[col].min()} max: {df_credit[col].max()}")
    
    
    
    

SK_ID_PREV mean: 1904503.589899727, min: 1000018 max: 2843496
SK_ID_CURR mean: 278324.20728888694, min: 100006 max: 456250
MONTHS_BALANCE mean: -34.52192061478338, min: -96 max: -1
AMT_BALANCE mean: 58300.15526185774, min: -420250.185 max: 1505902.185
AMT_CREDIT_LIMIT_ACTUAL mean: 153807.95740033622, min: 0 max: 1350000
AMT_DRAWINGS_ATM_CURRENT mean: 5961.324822059312, min: -6827.31 max: 2115000.0
AMT_DRAWINGS_CURRENT mean: 7433.388179202412, min: -6211.62 max: 2287098.315
AMT_DRAWINGS_OTHER_CURRENT mean: 288.16958242463346, min: 0.0 max: 1529847.0
AMT_DRAWINGS_POS_CURRENT mean: 2968.804847683069, min: 0.0 max: 2239274.16
AMT_INST_MIN_REGULARITY mean: 3540.204128515209, min: 0.0 max: 202882.005
AMT_PAYMENT_CURRENT mean: 10280.53770200115, min: 0.0 max: 4289207.445
AMT_PAYMENT_TOTAL_CURRENT mean: 7588.856739464025, min: 0.0 max: 4278315.69
AMT_RECEIVABLE_PRINCIPAL mean: 55965.87690549698, min: -423305.82 max: 1472316.795
AMT_RECIVABLE mean: 58088.81117667372, min: -420250.185 max: 14933

In [56]:
def process_credit_card_balance():
    df_credit = pd.read_csv("credit_card_balance.csv")
    credit_cat,new_cats = make_categorical(df_credit)
    credit_cat.drop(['SK_ID_PREV'],axis=1,inplace=True)
    credit_cat = credit_cat.groupby("SK_ID_CURR").agg(['min','max','mean'])
    credit_cat.columns = credit_cat.columns.map('_'.join)
    credit_cat.reset_index(inplace=True)
    credit_cat = make_days_positive(credit_cat)
    credit_cat = replace_max_erroneous_days(credit_cat)
    del df_credit
    return credit_cat

## Process Previous Applications Data

In [57]:
df_prev_app = pd.read_csv("previous_application.csv")
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
for col in df_prev_app.select_dtypes(include=numerics).columns:
    print(f"{col} mean: {df_prev_app[col].mean()}, min: {df_prev_app[col].min()} max: {df_prev_app[col].max()}")
    
    
    
    

SK_ID_PREV mean: 1923089.1353305625, min: 1000001 max: 2845382
SK_ID_CURR mean: 278357.1740986484, min: 100001 max: 456255
AMT_ANNUITY mean: 15955.120659450406, min: 0.0 max: 418058.145
AMT_APPLICATION mean: 175233.86036042805, min: 0.0 max: 6905160.0
AMT_CREDIT mean: 196114.02121797804, min: 0.0 max: 6905160.0
AMT_DOWN_PAYMENT mean: 6697.402138796564, min: -0.9 max: 3060045.0
AMT_GOODS_PRICE mean: 227847.27928334344, min: 0.0 max: 6905160.0
HOUR_APPR_PROCESS_START mean: 12.484181667738385, min: 0 max: 23
NFLAG_LAST_APPL_IN_DAY mean: 0.9964675185335532, min: 0 max: 1
RATE_DOWN_PAYMENT mean: 0.0796368154283738, min: -1.4978763414307847e-05 max: 1.0
RATE_INTEREST_PRIMARY mean: 0.18835688954982696, min: 0.03478125354187918 max: 1.0
RATE_INTEREST_PRIVILEGED mean: 0.7735025434991628, min: 0.3731501057082452 max: 1.0
DAYS_DECISION mean: -880.6796679946402, min: -2922 max: -1
SELLERPLACE_AREA mean: 313.95111524630977, min: -1 max: 4000000
CNT_PAYMENT mean: 16.0540815603274, min: 0.0 max: 84.0

In [58]:
def process_previous_apps():
    df_apps = pd.read_csv("previous_application.csv")
    df_apps.drop('SK_ID_PREV',axis=1,inplace=True)
    print(df_apps.shape)
    df_apps, new_cats = make_categorical(df_apps)
    print(df_apps.shape)
    df_apps['DAYS_FIRST_DRAWING'].replace(365243.0,df_apps['DAYS_FIRST_DRAWING'].median(),inplace=True)
    df_apps['DAYS_FIRST_DUE'].replace(365243.0,df_apps['DAYS_FIRST_DUE'].median(),inplace=True)
    df_apps['DAYS_LAST_DUE'].replace(365243.0,df_apps['DAYS_LAST_DUE'].median(),inplace=True)     
    df_apps['DAYS_TERMINATION'].replace(365243.0,df_apps['DAYS_TERMINATION'].median(),inplace=True)     
    df_apps['APP_CREDIT_PERC'] = df_apps['AMT_APPLICATION'] / df_apps['AMT_CREDIT']
    aggs = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum']
    }
    df_apps = make_days_positive(df_apps)
    df_apps = replace_max_erroneous_days(df_apps)
    for col in new_cats:
        aggs[col]=["mean"]
    df_apps_agg = df_apps.groupby("SK_ID_CURR").agg(aggs)
    df_apps_agg.columns = df_apps_agg.columns.map('_'.join)
    df_apps_agg.reset_index(inplace=True)
    del df_apps,new_cats
    return df_apps_agg
                                                     
    
    
    
    

In [59]:
!ls

'Clean Data .ipynb'
'Grab Categoricals(Label Encode).ipynb'
'Grab Categoricals(one_hot).ipynb'
 HomeCredit_columns_description.csv
 POS_CASH_balance.csv
'Test Simple Aggregates (compare one_hot vs. label_encode.ipynb'
 application_test.csv
 application_train.csv
 bureau.csv
 bureau_balance.csv
 credit_card_balance.csv
 df_test_one_hot.feather
 df_test_processed.feather
 df_train_one_hot.feather
 df_train_processed.feather
 df_train_test_app_cleaned.feather
 home-credit-default-risk.zip
 home_credit_explore.ipynb
 installments_payments.csv
 previous_application.csv
 sample_submission.csv
 xgboost_w_out_clean_null.pkl


In [60]:
df_cash = pd.read_csv("POS_CASH_balance.csv")

In [61]:
df_cash = pd.read_csv("POS_CASH_balance.csv")
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
for col in df_cash.select_dtypes(include=numerics).columns:
    print(f"{col} mean: {df_cash[col].mean()}, min: {df_cash[col].min()} max: {df_cash[col].max()}")
del df_cash
    
    
    

SK_ID_PREV mean: 1903216.5989568618, min: 1000001 max: 2843499
SK_ID_CURR mean: 278403.8633055631, min: 100001 max: 456255
MONTHS_BALANCE mean: -35.012587990550884, min: -96 max: -1
CNT_INSTALMENT mean: 17.089650453164907, min: 1.0 max: 92.0
CNT_INSTALMENT_FUTURE mean: 10.483840388897706, min: 0.0 max: 85.0
SK_DPD mean: 11.606928279139693, min: 0 max: 4231
SK_DPD_DEF mean: 0.654468423188131, min: 0 max: 3595


In [62]:
def process_cash_balance():
    df_cash = pd.read_csv("POS_CASH_balance.csv")
    df_cash, new_cats = make_categorical(df_cash)
    
    df_cash.drop("SK_ID_PREV",axis=1,inplace=True)
    aggs = {
        'MONTHS_BALANCE': ['max', 'mean', 'min'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for col in new_cats:
        aggs[col] = ['mean']
    
    
    
    
    
    df_cash = make_days_positive(df_cash)
    df_cash = replace_max_erroneous_days(df_cash)
    df_cash_agg = df_cash.groupby('SK_ID_CURR').agg(aggs)
    df_cash_agg.columns = df_cash_agg.columns.map('_'.join)
    df_cash_agg.reset_index(inplace=True)
    del df_cash
    return df_cash_agg

In [63]:
df_payments = pd.read_csv("installments_payments.csv")
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
for col in df_payments.select_dtypes(include=numerics).columns:
    print(f"{col} mean: {df_payments[col].mean()}, min: {df_payments[col].min()} max: {df_payments[col].max()}")
del df_payments

SK_ID_PREV mean: 1903364.9695492988, min: 1000001 max: 2843499
SK_ID_CURR mean: 278444.88173821557, min: 100001 max: 456255
NUM_INSTALMENT_VERSION mean: 0.8566373016127933, min: 0.0 max: 178.0
NUM_INSTALMENT_NUMBER mean: 18.87089583026623, min: 1 max: 277
DAYS_INSTALMENT mean: -1042.2699916011295, min: -2922.0 max: -1.0
DAYS_ENTRY_PAYMENT mean: -1051.1136840621016, min: -4921.0 max: -1.0
AMT_INSTALMENT mean: 17050.906988963343, min: 0.0 max: 3771487.845
AMT_PAYMENT mean: 17238.22324958399, min: 0.0 max: 3771487.845


In [64]:
def process_payments():
    df_payments = pd.read_csv("installments_payments.csv")
    df_payments,new_cats = make_categorical(df_payments)
    df_payments['PAYMENT_PERC'] = (df_payments['AMT_PAYMENT'] / 
                                   df_payments['AMT_INSTALMENT'])
    df_payments['PAYMENT_DIFF'] = (df_payments['AMT_INSTALMENT'] - 
                                   df_payments['AMT_PAYMENT'])
    aggs = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    
    for col in new_cats:
        aggs[col] = ['mean']
    df_payments = make_days_positive(df_payments)
    df_payments_agg = df_payments.groupby("SK_ID_CURR").agg(aggs)
    df_payments_agg.columns = df_payments_agg.columns.map('_'.join)
    df_payments_agg.reset_index(inplace=True)
    del df_payments
    return df_payments_agg
    
    

In [69]:
df_app_tot = process_train_test()
df_bureau = process_bureau()
df_credit = process_credit_card_balance()
df_payments = process_payments()
df_prev_apps = process_previous_apps()
df_cash_bal = process_cash_balance()


(1670214, 36)
(1670214, 163)


In [70]:
list_dfs = [df_app_tot,df_bureau,df_credit,df_payments,
            df_prev_apps,df_cash_bal]


In [71]:
num_cols = 0
for df in list_dfs:
    print(df.shape)
    num_cols+=df.shape[1]
print(f"Num Columns Should be: {num_cols}")

(356255, 125)
(305811, 57)
(103558, 82)
(339587, 20)
(338857, 174)
(337252, 17)
Num Columns Should be: 475


In [72]:
def join_dfs(df_app_tot):
    df_tot = df_app_tot.copy()
    print(df_tot.shape)
    for df in list_dfs[1:]:
        df_tot=df_tot.merge(df,how="left",on="SK_ID_CURR")
        print(f"Size after merging: {df_tot.shape}") 
    df_train = (df_tot.loc[(df_tot['TARGET']==1) |  
                            (df_tot['TARGET']==0)])
    df_test = df_tot.loc[df_tot['TARGET'].isnull()] 
    return df_train, df_test
        

In [73]:
df_train, df_test = join_dfs(df_app_tot) 

(356255, 125)
Size after merging: (356255, 181)
Size after merging: (356255, 262)
Size after merging: (356255, 281)
Size after merging: (356255, 454)
Size after merging: (356255, 470)


In [74]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

for col in df_train.select_dtypes(include=numerics).columns:
    print(f"{col} mean: {df_train[col].mean()}, min: {df_train[col].min()} max: {df_train[col].max()}")
    

SK_ID_CURR mean: 278180.51857657125, min: 100002 max: 456255
TARGET mean: 0.08072881945686496, min: 0.0 max: 1.0
NAME_CONTRACT_TYPE mean: 0.09521285417432222, min: 0 max: 1
CODE_GENDER mean: 0.3416690785045088, min: 0 max: 2
FLAG_OWN_CAR mean: 0.34010815873253314, min: 0 max: 1
FLAG_OWN_REALTY mean: 0.6936727466659729, min: 0 max: 1
CNT_CHILDREN mean: 0.4170517477423572, min: 0 max: 19
AMT_INCOME_TOTAL mean: 168797.9192969845, min: 25650.0 max: 117000000.0
AMT_CREDIT mean: 599025.9997057016, min: 45000.0 max: 4050000.0
AMT_ANNUITY mean: 27108.573909183444, min: 1615.5 max: 258025.5
AMT_GOODS_PRICE mean: 538396.2074288895, min: 40500.0 max: 4050000.0
NAME_TYPE_SUITE mean: 5.219992130362816, min: 0 max: 6
NAME_INCOME_TYPE mean: 4.670288217332063, min: 0 max: 7
NAME_EDUCATION_TYPE mean: 3.188272939829795, min: 0 max: 4
NAME_FAMILY_STATUS mean: 1.4723115595864864, min: 0 max: 5
NAME_HOUSING_TYPE mean: 1.2903896120789176, min: 0 max: 5
REGION_POPULATION_RELATIVE mean: 0.02086811205778947, m

CREDIT_TYPE_Consumer credit_mean mean: 0.7195213509101195, min: 0.0 max: 1.0
CREDIT_TYPE_Credit card_mean mean: 0.23965226299367356, min: 0.0 max: 1.0
CREDIT_TYPE_Interbank credit_mean mean: 6.325326734752483e-07, min: 0.0 max: 0.16666666666666666
CREDIT_TYPE_Loan for business development_mean mean: 0.001222049787799953, min: 0.0 max: 1.0
CREDIT_TYPE_Loan for purchase of shares (margin lending)_mean mean: 4.340147267230166e-06, min: 0.0 max: 0.5
CREDIT_TYPE_Loan for the purchase of equipment_mean mean: 1.4143606396003903e-05, min: 0.0 max: 1.0
CREDIT_TYPE_Loan for working capital replenishment_mean mean: 0.00022952731465481033, min: 0.0 max: 1.0
CREDIT_TYPE_Microloan_mean mean: 0.004207692378728733, min: 0.0 max: 1.0
CREDIT_TYPE_Mobile operator loan_mean mean: 2.3719975255321814e-07, min: 0.0 max: 0.0625
CREDIT_TYPE_Mortgage_mean mean: 0.014232926003697348, min: 0.0 max: 1.0
CREDIT_TYPE_Real estate loan_mean mean: 1.7852523820469593e-05, min: 0.0 max: 1.0
CREDIT_TYPE_Unknown type of lo

NAME_CASH_LOAN_PURPOSE_Everyday expenses_mean mean: 0.0012729553559678365, min: 0.0 max: 1.0
NAME_CASH_LOAN_PURPOSE_Furniture_mean mean: 0.00044900354669413805, min: 0.0 max: 1.0
NAME_CASH_LOAN_PURPOSE_Gasification / water supply_mean mean: 0.00016105788140942682, min: 0.0 max: 1.0
NAME_CASH_LOAN_PURPOSE_Hobby_mean mean: 1.9161474812520724e-05, min: 0.0 max: 0.5
NAME_CASH_LOAN_PURPOSE_Journey_mean mean: 0.0006352159261502655, min: 0.0 max: 1.0
NAME_CASH_LOAN_PURPOSE_Medicine_mean mean: 0.0011291639287442522, min: 0.0 max: 1.0
NAME_CASH_LOAN_PURPOSE_Money for a third person_mean mean: 1.552879706673718e-05, min: 0.0 max: 1.0
NAME_CASH_LOAN_PURPOSE_Other_mean mean: 0.007556055971786774, min: 0.0 max: 1.0
NAME_CASH_LOAN_PURPOSE_Payments on other loans_mean mean: 0.0006007476507012822, min: 0.0 max: 1.0
NAME_CASH_LOAN_PURPOSE_Purchase of electronic equipment_mean mean: 0.000626349880720313, min: 0.0 max: 1.0
NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_mean mean: 9.268724932667047e-06, 

## At this point I want to see if these mindlessly aggregated features produce a good result from there I will try and use my brain and come up with features that may push the needle a little bit more

In [75]:
feather.write_dataframe(df_train, "df_train_label_encode.feather")
feather.write_dataframe(df_test, "df_test_one_label_encode.feather")